## Brazilian E-Commerce Public Dataset by Olist

This is an anonymised Brazilian ecommerce public dataset of orders made at [Olist Store](https://olist.com/). It consists of around 100k orders from 2016 to 2018 made at multiple marketplaces in Brazil.<br>
Olist connects small businesses from all over Brazil with a single contract. Those merchants are able to sell their products through the Olist Store and ship them directly to the customers using Olist logistics partners. After a customer purchases the product from Olist Store a seller gets notified to fulfill that order. Once the customer receives the product, or the estimated delivery date is due, the customer gets a satisfaction survey by email where he can give a note for the purchase experience and write down some comments. <br><br>
Source: https://www.kaggle.com/olistbr/brazilian-ecommerce
<br><br>
### Goal 
The goal of this project is to predict the review score of an order.

In [1]:
import pandas as pd
import numpy as np

In [2]:
# Set general display options
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 200)

### Data Schema
Importing and joining the single datasets.

![](../images/data_schema.png)
source: https://www.kaggle.com/olistbr/brazilian-ecommerce

In [3]:
customers = pd.read_csv('../resources/Brazilian_retail/olist_customers_dataset.csv')
geolocations = pd.read_csv('../resources/Brazilian_retail/olist_geolocation_dataset.csv')
items = pd.read_csv('../resources/Brazilian_retail/olist_order_items_dataset.csv', 
                    parse_dates=['shipping_limit_date'])
payments = pd.read_csv('../resources/Brazilian_retail/olist_order_payments_dataset.csv')
reviews = pd.read_csv('../resources/Brazilian_retail/olist_order_reviews_dataset.csv',
                      parse_dates=['review_creation_date', 'review_answer_timestamp'])
orders = pd.read_csv('../resources/Brazilian_retail/olist_orders_dataset.csv',
                     parse_dates=['order_purchase_timestamp', 'order_approved_at',
                                  'order_delivered_carrier_date', 'order_delivered_customer_date',
                                  'order_estimated_delivery_date'])
products = pd.read_csv('../resources/Brazilian_retail/olist_products_dataset.csv')
sellers = pd.read_csv('../resources/Brazilian_retail/olist_sellers_dataset.csv')
categories = pd.read_csv('../resources/Brazilian_retail/product_category_name_translation.csv')

In [4]:
sellers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3095 entries, 0 to 3094
Data columns (total 4 columns):
 #   Column                  Non-Null Count  Dtype 
---  ------                  --------------  ----- 
 0   seller_id               3095 non-null   object
 1   seller_zip_code_prefix  3095 non-null   int64 
 2   seller_city             3095 non-null   object
 3   seller_state            3095 non-null   object
dtypes: int64(1), object(3)
memory usage: 96.8+ KB


In [5]:
geolocations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000163 entries, 0 to 1000162
Data columns (total 5 columns):
 #   Column                       Non-Null Count    Dtype  
---  ------                       --------------    -----  
 0   geolocation_zip_code_prefix  1000163 non-null  int64  
 1   geolocation_lat              1000163 non-null  float64
 2   geolocation_lng              1000163 non-null  float64
 3   geolocation_city             1000163 non-null  object 
 4   geolocation_state            1000163 non-null  object 
dtypes: float64(2), int64(1), object(2)
memory usage: 38.2+ MB


### Seller City Names 
Not each seller is using the same spelling of city names that's why in cases of ambiguity the first occurrence of a city name for the same zip code was used for all other sellers in the same location.

In [6]:
sellers['seller_zip_code_prefix'].nunique()

2246

In [7]:
# checking if a zip code of sellers can belong to various cities
zip_codes = sellers.groupby('seller_zip_code_prefix')[['seller_city']].nunique()
zip_code_ind = zip_codes[zip_codes['seller_city'] != 1].index
if len(zip_code_ind) > 0:
    display(sellers[sellers['seller_zip_code_prefix'].isin(zip_code_ind)] \
                .sort_values(by='seller_zip_code_prefix').head(10))
    
# there exist different spellings for the same city

,seller_id,seller_zip_code_prefix,seller_city,seller_state
1644,c13ef0cfbe42f190780f621ce81f2234,1207,sao paulo sp,SP
165,d6cd01c59123df02fc226eadbadb5f89,1207,sao paulo,SP
2536,0b18d63d0cd1d723567903fd34a07df2,5141,sp,SP
2573,8f2ce03f928b567e3d56181ae20ae952,5141,pirituba,SP
2074,72c5da29406b4234927b81855e7b64f6,7077,guarulhos,SP
1897,b64d51f0435e884e8de603b1655155ae,7077,garulhos,SP
802,596849622429351f47b32e6cae1055ff,8710,mogi das cruses,SP
1471,bc07d855eac23aab5ac8a120f21d16b2,8710,mogi das cruzes,SP
2056,01ed254b9ff8407dfb9d99ba1e17d923,8710,mogi das cruzes,SP
1382,ff1fb4c404b2efe68b03350a8dc24122,8710,mogi das cruzes,SP


In [8]:
# When a zip code relates to different cities (city names) it seems like to be due to spelling mistakes.
# Therefore, the first occurrence of a city name was taken when multiple options for a zip code are provided and 
# the remaining cities of that zip code were replaced by that.
for i in zip_code_ind:
    rows = sellers[sellers['seller_zip_code_prefix'] == i]['seller_city']
    city = rows.iloc[0]
    sellers.loc[rows.index, 'seller_city'] = sellers.loc[rows.index, 'seller_city'].map(lambda x: city)

In [9]:
sellers.duplicated().sum()

0

In [10]:
geolocations.duplicated().sum()

261831

In [11]:
geolocations.drop_duplicates(inplace=True)

In [12]:
geolocations['geolocation_zip_code_prefix'].nunique()

19015

In [13]:
geolocations['geolocation_zip_code_prefix'].value_counts()[:5]

38400    779
35500    751
11680    727
11740    678
36400    627
Name: geolocation_zip_code_prefix, dtype: int64

In [14]:
geolocations[geolocations['geolocation_zip_code_prefix'] == 38400][:10]

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
645450,38400,-18.915960,-48.278974,uberlandia,MG
645456,38400,-18.900442,-48.260759,uberlandia,MG
645460,38400,-18.913015,-48.262132,uberlandia,MG
645461,38400,-18.922381,-48.282111,uberlandia,MG
645471,38400,-18.922160,-48.271125,uberlandia,MG
645474,38400,-18.915501,-48.263653,uberlândia,MG
645479,38400,-18.921864,-48.295130,uberlandia,MG
645494,38400,-18.903228,-48.274547,uberlândia,MG
645501,38400,-18.903908,-48.273556,uberlandia,MG
645511,38400,-18.923919,-48.278409,uberlandia,MG


### Geolocation
For each zip code was only one entry kept because due to the data schema description the customer and seller datasets shall be joined with the geolocation dataset through the `zip_code_prefix`. That means, that the longitude and latitude features won't represent the actual value for each customer and seller.

In [15]:
# Because geolocation data points should be merged with the zip code entries, only one latitude and longitude value 
# per zip code will be kept.
geolocations_droplist = geolocations[geolocations['geolocation_zip_code_prefix'].duplicated()].index
geolocations_droplist.shape

(719317,)

In [16]:
geolocations.drop(geolocations_droplist, inplace=True)

In [17]:
# merge seller and geolocation datasets
seller_geo = pd.merge(left=geolocations.drop(['geolocation_city', 'geolocation_state'], axis=1), 
                      right=sellers, how='right', 
                      left_on='geolocation_zip_code_prefix',
                      right_on='seller_zip_code_prefix')

In [18]:
sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP


In [19]:
geolocations.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,geolocation_city,geolocation_state
0,1037,-23.545621,-46.639292,sao paulo,SP
1,1046,-23.546081,-46.644820,sao paulo,SP
3,1041,-23.544392,-46.639499,sao paulo,SP
4,1035,-23.541578,-46.641607,sao paulo,SP
5,1012,-23.547762,-46.635361,são paulo,SP


In [20]:
seller_geo.head()

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,1041.0,-23.544392,-46.639499,e5cbe890e679490127e9a390b46bbd20,1041,sao paulo,SP
1,1035.0,-23.541578,-46.641607,1d503743d2526f03f0c2c89540ee008c,1035,sao paulo,SP
2,1039.0,-23.541883,-46.639919,2d34636518ba88f5349b732fcf8ba2e4,1039,sao paulo,SP
3,1042.0,-23.544874,-46.640303,b6c6854d4d92a5f6f46be8869da3fa1a,1042,sao paulo,SP
4,1040.0,-23.540317,-46.637355,784ba75dd9d20200c4caed3d7a77141a,1040,sao paulo,SP


In [21]:
seller_geo['geolocation_zip_code_prefix'].isnull().sum()

7

In [22]:
seller_geo[seller_geo['geolocation_zip_code_prefix'].isnull()]

,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,seller_id,seller_zip_code_prefix,seller_city,seller_state
3088,NaN,NaN,NaN,5962468f885ea01a1b6a97a218797b0a,82040,curitiba,PR
3089,NaN,NaN,NaN,2aafae69bf4c41fbd94053d9413e87ee,91901,porto alegre,RS
3090,NaN,NaN,NaN,2a50b7ee5aebecc6fd0ff9784a4747d6,72580,brasilia,DF
3091,NaN,NaN,NaN,2e90cb1677d35cfe24eef47d441b7c87,2285,sao paulo,SP
3092,NaN,NaN,NaN,0b3f27369a4d8df98f7eb91077e438ac,7412,aruja,SP
3093,NaN,NaN,NaN,42bde9fef835393bb8a8849cb6b7f245,71551,brasilia,DF
3094,NaN,NaN,NaN,870d0118f7a9d85960f29ad89d5d989a,37708,pocos de caldas,MG


In [23]:
customers.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 5 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   customer_id               99441 non-null  object
 1   customer_unique_id        99441 non-null  object
 2   customer_zip_code_prefix  99441 non-null  int64 
 3   customer_city             99441 non-null  object
 4   customer_state            99441 non-null  object
dtypes: int64(1), object(4)
memory usage: 3.8+ MB


In [24]:
customers.duplicated().sum()

0

### Customer City Names 
The same problem with the city names for the sellers happened to the customers. Consequently, the same procedure was applied here too.

In [25]:
# checking if a zip code of customers can belong to various cities
zip_codes = customers.groupby('customer_zip_code_prefix')[['customer_city']].nunique()
zip_code_ind = zip_codes[zip_codes['customer_city'] != 1].index
if len(zip_code_ind) > 0:
    display(customers[customers['customer_zip_code_prefix'] == zip_code_ind[0]])

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state
11189,502de0f9e9835ab231c2851acd163376,6d004368588f6fee399cff4bfd5bf7eb,6806,embu das artes,SP
24446,ad587a1071e2eb39139bdf6c7e49d90b,83d0c398734ab60891d351269aca2aa0,6806,embu das artes,SP
58777,8b8e02be8fef10b7c6e2d210dbcfba7c,ed05a1bbf9f8816af23fcc68084bb87d,6806,embu,SP
99135,2ed1b5c01561bcf39cc2f1566a8ac9f1,ed05a1bbf9f8816af23fcc68084bb87d,6806,embu,SP


In [26]:
# The same procedure like for sellers to handle different city names for an identical zip code for customers 
# under the assumption that this scenario occurs due to spelling errors. 
for i in zip_code_ind:
    rows = customers[customers['customer_zip_code_prefix'] == i]['customer_city']
    city = rows.iloc[0]
    customers.loc[rows.index, 'customer_city'] = customers.loc[rows.index, 'customer_city'].map(lambda x: city)

In [27]:
# merging customers and geolocations datasets
cust_geo = pd.merge(left=customers, right=geolocations.drop(['geolocation_city', 'geolocation_state'], axis=1), 
                    how='left',
                    left_on='customer_zip_code_prefix',
                    right_on='geolocation_zip_code_prefix')

In [28]:
cust_geo.head()

,customer_id,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng
0,06b8999e2fba1a1fbc88172c00ba8bc7,861eff4711a542e4b93843c6dd7febb0,14409,franca,SP,14409.0,-20.509897,-47.397866
1,18955e83d337fd6b2def6b18a428ac77,290c77bc529b7ac935b93aa66c333dc3,9790,sao bernardo do campo,SP,9790.0,-23.726853,-46.545746
2,4e7b3e00288586ebd08712fdd0374a03,060e732b5b29e8181a18229c7b0b2b5e,1151,sao paulo,SP,1151.0,-23.527788,-46.660310
3,b2b6027bc5c5109e529d4dc6358b12c3,259dac757896d24d7702b9acbbff3f3c,8775,mogi das cruzes,SP,8775.0,-23.496930,-46.185352
4,4f2d8ab171c80ec8364f7c12e35b23ad,345ecd01c38d18a9036ed96c73b8d066,13056,campinas,SP,13056.0,-22.987222,-47.151073


In [29]:
orders.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 99441 entries, 0 to 99440
Data columns (total 8 columns):
 #   Column                         Non-Null Count  Dtype         
---  ------                         --------------  -----         
 0   order_id                       99441 non-null  object        
 1   customer_id                    99441 non-null  object        
 2   order_status                   99441 non-null  object        
 3   order_purchase_timestamp       99441 non-null  datetime64[ns]
 4   order_approved_at              99281 non-null  datetime64[ns]
 5   order_delivered_carrier_date   97658 non-null  datetime64[ns]
 6   order_delivered_customer_date  96476 non-null  datetime64[ns]
 7   order_estimated_delivery_date  99441 non-null  datetime64[ns]
dtypes: datetime64[ns](5), object(3)
memory usage: 6.1+ MB


In [30]:
orders.duplicated().sum()

0

In [31]:
# merging of following datasets: customers/geolocations with orders
ord_cust_geo = pd.merge(left=orders, right=cust_geo, 
                        how='left',
                        on='customer_id')

In [32]:
ord_cust_geo.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,3149.0,-23.574809,-46.587471
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,47813.0,-12.169860,-44.988369
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,75265.0,-16.746337,-48.514624
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15,7c142cf63193a1473d2e66489a9ae977,59296,sao goncalo do amarante,RN,59296.0,-5.767733,-35.275467
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26,72632f0f9dd73dfee390c9b22eb56dd6,9195,santo andre,SP,9195.0,-23.675037,-46.524784


In [33]:
reviews.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 7 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   review_id                100000 non-null  object        
 1   order_id                 100000 non-null  object        
 2   review_score             100000 non-null  int64         
 3   review_comment_title     11715 non-null   object        
 4   review_comment_message   41753 non-null   object        
 5   review_creation_date     100000 non-null  datetime64[ns]
 6   review_answer_timestamp  100000 non-null  datetime64[ns]
dtypes: datetime64[ns](2), int64(1), object(4)
memory usage: 5.3+ MB


In [34]:
# merging of following datasets: orders/customers/geolocations with reviews
ord_cust_geo_rev = pd.merge(left=ord_cust_geo, right=reviews,
                            how='left', 
                            on='order_id')

In [35]:
ord_cust_geo_rev.shape

(100000, 21)

In [36]:
# checking if an order occurs at least twice
multi_reviews = ord_cust_geo_rev[ord_cust_geo_rev['order_id'].duplicated()]
multi_reviews.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
406,ac7a92560a9e99f0a0ab4988102f491b,6abf781c5304890987047e9aa239caa8,delivered,2018-03-04 15:58:09,2018-03-04 16:10:35,2018-03-06 17:14:59,2018-03-22 23:44:33,2018-03-23,a176838867d89ed62981cdf02b7ca3e1,86040,londrina,PR,86040.0,-23.334722,-51.145223,5ac791169055b4994a395a442e3956f6,1,NaN,"Comprei pensando que ia vir conforme a foto, v...",2018-03-23,2018-03-24 12:57:14
557,7845a2492ab1b4f2cf3d56c7b8da1446,386e96046ce0e77e5c9a9a4e818da93c,delivered,2018-01-18 12:35:45,2018-01-18 12:56:40,2018-01-18 23:42:44,2018-01-23 21:51:48,2018-02-09,1175e95fb47ddff9de6b2b06188f7e0d,81560,curitiba,PR,81560.0,-25.496853,-49.209023,8566d94f7bf3bedbb4ca6a29725fc382,5,NaN,chegou antes do prometido,2018-01-27,2018-01-30 19:18:27
616,714fb133a6730ab81fa1d3c1b2007291,e3fe72696c4713d64d3c10afe71e75ed,canceled,2018-01-26 21:34:08,2018-01-26 21:58:39,2018-01-29 22:33:25,NaT,2018-02-22,c4ebedb09beb89cc0314c5c0c33f8053,9961,diadema,SP,9961.0,-23.716539,-46.601640,105facb42b5b9147996f0b1b6fbf7b80,1,NaN,Holá!\r\nNão recebi o produto é nem um telefon...,2018-02-24,2018-02-27 00:13:54
672,acbe07f22f29ad7e5a78f30008cc6ec7,b4afeb58ac51bc903c5362286c6a5cfe,delivered,2017-11-18 14:33:04,2017-11-18 19:06:16,2017-11-27 18:34:31,2017-11-29 17:58:43,2017-12-05,6457b2cb5462c1217e8e6d36b0ff39d2,5842,sao paulo,SP,5842.0,-23.649263,-46.741918,a3c3bd7b9583c15aa8b72bbb94e8744d,5,NaN,NaN,2017-11-25,2017-11-25 21:21:54
819,c761a8b74f1e876bc5efc4186f720e27,a8ed087e25b26fb64cbb99033eb243a7,delivered,2017-04-03 19:07:32,2017-04-03 19:22:45,2017-04-05 12:42:51,2017-04-11 14:25:21,2017-05-15,3702265e16e5246cff7462ea675f6d90,29313,cachoeiro de itapemirim,ES,29313.0,-20.825999,-41.136452,8edd42a7c561e11de943594bfae347e5,1,NaN,Realizei a compra de 2 colchas de solteiro e s...,2017-04-12,2017-04-13 01:14:56


In [37]:
# displaying one order which occurs more than once
ord_cust_geo_rev[ord_cust_geo_rev['order_id'] == multi_reviews['order_id'].iloc[0]]

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
405,ac7a92560a9e99f0a0ab4988102f491b,6abf781c5304890987047e9aa239caa8,delivered,2018-03-04 15:58:09,2018-03-04 16:10:35,2018-03-06 17:14:59,2018-03-22 23:44:33,2018-03-23,a176838867d89ed62981cdf02b7ca3e1,86040,londrina,PR,86040.0,-23.334722,-51.145223,1ce9388b1a3ad76f5f1a64f208a90fce,1,NaN,O prazo era até dia 23/03 e até o momento não ...,2018-03-25,2018-03-25 21:40:11
406,ac7a92560a9e99f0a0ab4988102f491b,6abf781c5304890987047e9aa239caa8,delivered,2018-03-04 15:58:09,2018-03-04 16:10:35,2018-03-06 17:14:59,2018-03-22 23:44:33,2018-03-23,a176838867d89ed62981cdf02b7ca3e1,86040,londrina,PR,86040.0,-23.334722,-51.145223,5ac791169055b4994a395a442e3956f6,1,NaN,"Comprei pensando que ia vir conforme a foto, v...",2018-03-23,2018-03-24 12:57:14


In [38]:
# checking which features differ
mask_multi_review = ord_cust_geo_rev.loc[405] == ord_cust_geo_rev.loc[406]
mask_multi_review

order_id                          True
customer_id                       True
order_status                      True
order_purchase_timestamp          True
order_approved_at                 True
order_delivered_carrier_date      True
order_delivered_customer_date     True
order_estimated_delivery_date     True
customer_unique_id                True
customer_zip_code_prefix          True
customer_city                     True
customer_state                    True
geolocation_zip_code_prefix       True
geolocation_lat                   True
geolocation_lng                   True
review_id                        False
review_score                      True
review_comment_title             False
review_comment_message           False
review_creation_date             False
review_answer_timestamp          False
dtype: bool

In [39]:
ord_cust_geo_rev.loc[[405, 406], ~mask_multi_review]

,review_id,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
405,1ce9388b1a3ad76f5f1a64f208a90fce,NaN,O prazo era até dia 23/03 e até o momento não ...,2018-03-25,2018-03-25 21:40:11
406,5ac791169055b4994a395a442e3956f6,NaN,"Comprei pensando que ia vir conforme a foto, v...",2018-03-23,2018-03-24 12:57:14


In [40]:
payments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 103886 entries, 0 to 103885
Data columns (total 5 columns):
 #   Column                Non-Null Count   Dtype  
---  ------                --------------   -----  
 0   order_id              103886 non-null  object 
 1   payment_sequential    103886 non-null  int64  
 2   payment_type          103886 non-null  object 
 3   payment_installments  103886 non-null  int64  
 4   payment_value         103886 non-null  float64
dtypes: float64(1), int64(2), object(2)
memory usage: 4.0+ MB


In [41]:
payments.duplicated().sum()

0

In [42]:
# additional payments (sequential payments)
additional_payments = payments[payments['order_id'].duplicated()].shape[0]
additional_payments

4446

In [43]:
# seeing structure of first order_id with more than one payment
order_id_double_payments = payments[payments['order_id'].duplicated()]['order_id'].iloc[0]
payments[payments['order_id'] == order_id_double_payments]

,order_id,payment_sequential,payment_type,payment_installments,payment_value
385,683bf306149bb869980b68d48a1bd6ab,2,voucher,1,30.00
1456,683bf306149bb869980b68d48a1bd6ab,1,credit_card,1,8.58


In [44]:
# in comparison with a single payment
order_id_one_payment = payments[~payments['order_id'].duplicated()]['order_id'].iloc[:5]
payments[payments['order_id'].isin(order_id_one_payment)]

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [45]:
# merging of following datasets: orders/customers/geolocations/reviews with payments
ord_cust_geo_rev_pay = pd.merge(left=ord_cust_geo_rev, right=payments,
                                how='left', 
                                on='order_id')

In [46]:
# one order was done by 29 single payments
sorted(ord_cust_geo_rev_pay['payment_sequential'].unique())

[1.0,
 2.0,
 3.0,
 4.0,
 5.0,
 6.0,
 7.0,
 8.0,
 9.0,
 10.0,
 11.0,
 12.0,
 13.0,
 14.0,
 15.0,
 16.0,
 17.0,
 18.0,
 19.0,
 20.0,
 21.0,
 22.0,
 23.0,
 24.0,
 25.0,
 26.0,
 27.0,
 28.0,
 29.0,
 nan]

In [47]:
ord_cust_geo_rev_pay['payment_sequential'].isnull().sum()

1

In [48]:
multi_reviews

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
406,ac7a92560a9e99f0a0ab4988102f491b,6abf781c5304890987047e9aa239caa8,delivered,2018-03-04 15:58:09,2018-03-04 16:10:35,2018-03-06 17:14:59,2018-03-22 23:44:33,2018-03-23,a176838867d89ed62981cdf02b7ca3e1,86040,londrina,PR,86040.0,-23.334722,-51.145223,5ac791169055b4994a395a442e3956f6,1,NaN,"Comprei pensando que ia vir conforme a foto, v...",2018-03-23,2018-03-24 12:57:14
557,7845a2492ab1b4f2cf3d56c7b8da1446,386e96046ce0e77e5c9a9a4e818da93c,delivered,2018-01-18 12:35:45,2018-01-18 12:56:40,2018-01-18 23:42:44,2018-01-23 21:51:48,2018-02-09,1175e95fb47ddff9de6b2b06188f7e0d,81560,curitiba,PR,81560.0,-25.496853,-49.209023,8566d94f7bf3bedbb4ca6a29725fc382,5,NaN,chegou antes do prometido,2018-01-27,2018-01-30 19:18:27
616,714fb133a6730ab81fa1d3c1b2007291,e3fe72696c4713d64d3c10afe71e75ed,canceled,2018-01-26 21:34:08,2018-01-26 21:58:39,2018-01-29 22:33:25,NaT,2018-02-22,c4ebedb09beb89cc0314c5c0c33f8053,9961,diadema,SP,9961.0,-23.716539,-46.601640,105facb42b5b9147996f0b1b6fbf7b80,1,NaN,Holá!\r\nNão recebi o produto é nem um telefon...,2018-02-24,2018-02-27 00:13:54
672,acbe07f22f29ad7e5a78f30008cc6ec7,b4afeb58ac51bc903c5362286c6a5cfe,delivered,2017-11-18 14:33:04,2017-11-18 19:06:16,2017-11-27 18:34:31,2017-11-29 17:58:43,2017-12-05,6457b2cb5462c1217e8e6d36b0ff39d2,5842,sao paulo,SP,5842.0,-23.649263,-46.741918,a3c3bd7b9583c15aa8b72bbb94e8744d,5,NaN,NaN,2017-11-25,2017-11-25 21:21:54
819,c761a8b74f1e876bc5efc4186f720e27,a8ed087e25b26fb64cbb99033eb243a7,delivered,2017-04-03 19:07:32,2017-04-03 19:22:45,2017-04-05 12:42:51,2017-04-11 14:25:21,2017-05-15,3702265e16e5246cff7462ea675f6d90,29313,cachoeiro de itapemirim,ES,29313.0,-20.825999,-41.136452,8edd42a7c561e11de943594bfae347e5,1,NaN,Realizei a compra de 2 colchas de solteiro e s...,2017-04-12,2017-04-13 01:14:56
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99403,66f943fd426ac8716ef6b6b97ace8bd8,592ef413ebc8f70a779b9282d2ea6020,delivered,2018-04-19 08:26:03,2018-04-19 08:35:19,2018-04-27 10:56:00,2018-05-04 19:51:06,2018-05-09,d6df1c8cb71c59d9b52d828dc6d0f5b3,71926,brasilia,DF,71926.0,-15.844322,-48.025902,090aa2980b0aea78cd1ae73447d7d55b,4,NaN,NaN,2018-05-10,2018-05-11 10:52:52
99578,76de574e14e58baded25a12dce8075c1,3dc99b2fcf41c0898cfc7f38c2fefe5e,delivered,2017-06-18 10:26:33,2017-06-18 10:42:57,2017-06-19 13:45:40,2017-06-22 16:04:05,2017-07-11,970e5961383615662043afa2d9c68802,30880,belo horizonte,MG,30880.0,-19.905684,-44.021399,0234abe22674d1798d6c3f95aea9033b,5,NaN,Bom demias!!! adoro o baratheon.,2017-06-25,2017-07-01 11:39:23
99604,2fcdb0c004a2a538d3dd724ea76916bd,cf826be53b1fe4a39bc33c7f5f827835,delivered,2017-11-22 11:39:00,2017-11-22 11:49:05,2017-11-23 22:58:51,2017-12-15 00:23:25,2017-12-18,d92462d83762d5e5c112bccc344119e6,65940,grajau,MA,65940.0,-5.814610,-46.142801,4c4bc2df56b20a1bc3749622240a427f,5,NaN,Produto exelente,2017-12-15,2017-12-16 06:50:19
99640,dbfa3d8c05cc5d84083fc259a9feb5bc,92fc29e76e4da7c1b189ab092d4c3756,delivered,2018-01-04 22:57:14,2018-01-06 02:08:45,2018-01-13 00:51:54,2018-01-22 21:16:37,2018-02-01,8901b2526349d372dd0bdf8d77af656d,79823,dourados,MS,79823.0,-22.217034,-54.832386,3052a79d76a1932ff98c990a59230f8d,5,NaN,NaN,2018-01-23,2018-01-24 00:17:19


In [49]:
# finding orders with multi reviews and different sequential payments
ord_cust_geo_rev_pay[(ord_cust_geo_rev_pay['order_id'].duplicated()) & 
                     (ord_cust_geo_rev_pay['payment_sequential'] != 1.) &
                     (ord_cust_geo_rev_pay['order_id'].isin(multi_reviews['order_id']))].head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,payment_sequential,payment_type,payment_installments,payment_value
3036,9c54a1816e487456bb7eb5082b9823ff,1d48a5dcde374b9d63b2c29e6af97cb8,delivered,2017-01-17 14:23:54,2017-01-17 14:30:16,2017-01-18 08:27:58,2017-02-08 08:43:59,2017-03-02,1d2435aa3b858d45c707c9fc25e18779,21760,rio de janeiro,RJ,21760.0,-22.888836,-43.415253,b5c66480c76e304e904cf9c16a883113,5,NaN,NaN,2017-01-31,2017-02-01 10:04:17,5.0,voucher,1.0,9.37
3037,9c54a1816e487456bb7eb5082b9823ff,1d48a5dcde374b9d63b2c29e6af97cb8,delivered,2017-01-17 14:23:54,2017-01-17 14:30:16,2017-01-18 08:27:58,2017-02-08 08:43:59,2017-03-02,1d2435aa3b858d45c707c9fc25e18779,21760,rio de janeiro,RJ,21760.0,-22.888836,-43.415253,b5c66480c76e304e904cf9c16a883113,5,NaN,NaN,2017-01-31,2017-02-01 10:04:17,8.0,voucher,1.0,8.57
3038,9c54a1816e487456bb7eb5082b9823ff,1d48a5dcde374b9d63b2c29e6af97cb8,delivered,2017-01-17 14:23:54,2017-01-17 14:30:16,2017-01-18 08:27:58,2017-02-08 08:43:59,2017-03-02,1d2435aa3b858d45c707c9fc25e18779,21760,rio de janeiro,RJ,21760.0,-22.888836,-43.415253,b5c66480c76e304e904cf9c16a883113,5,NaN,NaN,2017-01-31,2017-02-01 10:04:17,6.0,voucher,1.0,17.95
3039,9c54a1816e487456bb7eb5082b9823ff,1d48a5dcde374b9d63b2c29e6af97cb8,delivered,2017-01-17 14:23:54,2017-01-17 14:30:16,2017-01-18 08:27:58,2017-02-08 08:43:59,2017-03-02,1d2435aa3b858d45c707c9fc25e18779,21760,rio de janeiro,RJ,21760.0,-22.888836,-43.415253,b5c66480c76e304e904cf9c16a883113,5,NaN,NaN,2017-01-31,2017-02-01 10:04:17,4.0,voucher,1.0,14.20
3040,9c54a1816e487456bb7eb5082b9823ff,1d48a5dcde374b9d63b2c29e6af97cb8,delivered,2017-01-17 14:23:54,2017-01-17 14:30:16,2017-01-18 08:27:58,2017-02-08 08:43:59,2017-03-02,1d2435aa3b858d45c707c9fc25e18779,21760,rio de janeiro,RJ,21760.0,-22.888836,-43.415253,b5c66480c76e304e904cf9c16a883113,5,NaN,NaN,2017-01-31,2017-02-01 10:04:17,7.0,voucher,1.0,4.82


In [50]:
# following order was done by partial payments and has reviewed twice (see review score and 
# review_answer_timestamp)
ord_cust_geo_rev_pay[ord_cust_geo_rev_pay['order_id'] == '9c54a1816e487456bb7eb5082b9823ff']

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix,geolocation_lat,geolocation_lng,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,payment_sequential,payment_type,payment_installments,payment_value
3035,9c54a1816e487456bb7eb5082b9823ff,1d48a5dcde374b9d63b2c29e6af97cb8,delivered,2017-01-17 14:23:54,2017-01-17 14:30:16,2017-01-18 08:27:58,2017-02-08 08:43:59,2017-03-02,1d2435aa3b858d45c707c9fc25e18779,21760,rio de janeiro,RJ,21760.0,-22.888836,-43.415253,b5c66480c76e304e904cf9c16a883113,5,NaN,NaN,2017-01-31,2017-02-01 10:04:17,1.0,credit_card,1.0,2.01
3036,9c54a1816e487456bb7eb5082b9823ff,1d48a5dcde374b9d63b2c29e6af97cb8,delivered,2017-01-17 14:23:54,2017-01-17 14:30:16,2017-01-18 08:27:58,2017-02-08 08:43:59,2017-03-02,1d2435aa3b858d45c707c9fc25e18779,21760,rio de janeiro,RJ,21760.0,-22.888836,-43.415253,b5c66480c76e304e904cf9c16a883113,5,NaN,NaN,2017-01-31,2017-02-01 10:04:17,5.0,voucher,1.0,9.37
3037,9c54a1816e487456bb7eb5082b9823ff,1d48a5dcde374b9d63b2c29e6af97cb8,delivered,2017-01-17 14:23:54,2017-01-17 14:30:16,2017-01-18 08:27:58,2017-02-08 08:43:59,2017-03-02,1d2435aa3b858d45c707c9fc25e18779,21760,rio de janeiro,RJ,21760.0,-22.888836,-43.415253,b5c66480c76e304e904cf9c16a883113,5,NaN,NaN,2017-01-31,2017-02-01 10:04:17,8.0,voucher,1.0,8.57
3038,9c54a1816e487456bb7eb5082b9823ff,1d48a5dcde374b9d63b2c29e6af97cb8,delivered,2017-01-17 14:23:54,2017-01-17 14:30:16,2017-01-18 08:27:58,2017-02-08 08:43:59,2017-03-02,1d2435aa3b858d45c707c9fc25e18779,21760,rio de janeiro,RJ,21760.0,-22.888836,-43.415253,b5c66480c76e304e904cf9c16a883113,5,NaN,NaN,2017-01-31,2017-02-01 10:04:17,6.0,voucher,1.0,17.95
3039,9c54a1816e487456bb7eb5082b9823ff,1d48a5dcde374b9d63b2c29e6af97cb8,delivered,2017-01-17 14:23:54,2017-01-17 14:30:16,2017-01-18 08:27:58,2017-02-08 08:43:59,2017-03-02,1d2435aa3b858d45c707c9fc25e18779,21760,rio de janeiro,RJ,21760.0,-22.888836,-43.415253,b5c66480c76e304e904cf9c16a883113,5,NaN,NaN,2017-01-31,2017-02-01 10:04:17,4.0,voucher,1.0,14.20
3040,9c54a1816e487456bb7eb5082b9823ff,1d48a5dcde374b9d63b2c29e6af97cb8,delivered,2017-01-17 14:23:54,2017-01-17 14:30:16,2017-01-18 08:27:58,2017-02-08 08:43:59,2017-03-02,1d2435aa3b858d45c707c9fc25e18779,21760,rio de janeiro,RJ,21760.0,-22.888836,-43.415253,b5c66480c76e304e904cf9c16a883113,5,NaN,NaN,2017-01-31,2017-02-01 10:04:17,7.0,voucher,1.0,4.82
3041,9c54a1816e487456bb7eb5082b9823ff,1d48a5dcde374b9d63b2c29e6af97cb8,delivered,2017-01-17 14:23:54,2017-01-17 14:30:16,2017-01-18 08:27:58,2017-02-08 08:43:59,2017-03-02,1d2435aa3b858d45c707c9fc25e18779,21760,rio de janeiro,RJ,21760.0,-22.888836,-43.415253,b5c66480c76e304e904cf9c16a883113,5,NaN,NaN,2017-01-31,2017-02-01 10:04:17,3.0,voucher,1.0,14.60
3042,9c54a1816e487456bb7eb5082b9823ff,1d48a5dcde374b9d63b2c29e6af97cb8,delivered,2017-01-17 14:23:54,2017-01-17 14:30:16,2017-01-18 08:27:58,2017-02-08 08:43:59,2017-03-02,1d2435aa3b858d45c707c9fc25e18779,21760,rio de janeiro,RJ,21760.0,-22.888836,-43.415253,b5c66480c76e304e904cf9c16a883113,5,NaN,NaN,2017-01-31,2017-02-01 10:04:17,2.0,voucher,1.0,14.60
3043,9c54a1816e487456bb7eb5082b9823ff,1d48a5dcde374b9d63b2c29e6af97cb8,delivered,2017-01-17 14:23:54,2017-01-17 14:30:16,2017-01-18 08:27:58,2017-02-08 08:43:59,2017-03-02,1d2435aa3b858d45c707c9fc25e18779,21760,rio de janeiro,RJ,21760.0,-22.888836,-43.415253,7c46b82b29a9caaeb06d6c03bbcecab4,4,NaN,NaN,2017-02-09,2017-02-10 09:36:39,1.0,credit_card,1.0,2.01
3044,9c54a1816e487456bb7eb5082b9823ff,1d48a5dcde374b9d63b2c29e6af97cb8,delivered,2017-01-17 14:23:54,2017-01-17 14:30:16,2017-01-18 08:27:58,2017-02-08 08:43:59,2017-03-02,1d2435aa3b858d45c707c9fc25e18779,21760,rio de janeiro,RJ,21760.0,-22.888836,-43.415253,7c46b82b29a9caaeb06d6c03bbcecab4,4,NaN,NaN,2017-02-09,2017-02-10

In [51]:
items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112650 entries, 0 to 112649
Data columns (total 7 columns):
 #   Column               Non-Null Count   Dtype         
---  ------               --------------   -----         
 0   order_id             112650 non-null  object        
 1   order_item_id        112650 non-null  int64         
 2   product_id           112650 non-null  object        
 3   seller_id            112650 non-null  object        
 4   shipping_limit_date  112650 non-null  datetime64[ns]
 5   price                112650 non-null  float64       
 6   freight_value        112650 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(3)
memory usage: 6.0+ MB


In [52]:
items.duplicated().sum()

0

In [53]:
items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [54]:
# figuring out why some orders are duplicated 
order_id_multi_item = items[items['order_id'].duplicated()]['order_id'].iloc[0]
items[items['order_id'] == order_id_multi_item]

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
13,0008288aa423d2a3f00fcb17cd7d8719,1,368c6c730842d78016ad823897a372db,1f50f920176fa81dab994f9023523100,2018-02-21 02:55:52,49.9,13.37
14,0008288aa423d2a3f00fcb17cd7d8719,2,368c6c730842d78016ad823897a372db,1f50f920176fa81dab994f9023523100,2018-02-21 02:55:52,49.9,13.37


In [55]:
# conclusion: order_item_id is changing
items.iloc[13,:] == items.iloc[14,:]

order_id                True
order_item_id          False
product_id              True
seller_id               True
shipping_limit_date     True
price                   True
freight_value           True
dtype: bool

In [56]:
# merging of following datasets: orders/customers/geolocations/reviews/payments with items
ord_cust_geo_rev_pay_item = pd.merge(left=ord_cust_geo_rev_pay, right=items,
                                     how='left',
                                     on='order_id')

In [57]:
ord_cust_geo_rev_pay_item.shape

(119151, 31)

In [58]:
products.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 32951 entries, 0 to 32950
Data columns (total 9 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   product_id                  32951 non-null  object 
 1   product_category_name       32341 non-null  object 
 2   product_name_lenght         32341 non-null  float64
 3   product_description_lenght  32341 non-null  float64
 4   product_photos_qty          32341 non-null  float64
 5   product_weight_g            32949 non-null  float64
 6   product_length_cm           32949 non-null  float64
 7   product_height_cm           32949 non-null  float64
 8   product_width_cm            32949 non-null  float64
dtypes: float64(7), object(2)
memory usage: 2.3+ MB


In [59]:
products.duplicated().sum()

0

In [60]:
products.nunique()

product_id                    32951
product_category_name            73
product_name_lenght              66
product_description_lenght     2960
product_photos_qty               19
product_weight_g               2204
product_length_cm                99
product_height_cm               102
product_width_cm                 95
dtype: int64

In [61]:
# merging of following datasets: orders/customers/geolocations/reviews/payments/items with products
ord_cust_geo_rev_pay_item_prod = pd.merge(left=ord_cust_geo_rev_pay_item, right=products,
                                          how='left',
                                          on='product_id')

In [62]:
# merging of following datasets: orders/customers/geolocations/reviews/payments/items/products with 
# sellers/geolocations
ord_cust_geo_rev_pay_item_prod_sell = pd.merge(left=ord_cust_geo_rev_pay_item_prod,
                                               right=seller_geo,
                                               how='left',
                                               on='seller_id',
                                               suffixes=('_customer', '_seller'))

In [63]:
ord_cust_geo_rev_pay_item_prod_sell.shape

(119151, 45)

In [64]:
ord_cust_geo_rev_pay_item_prod_sell.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,geolocation_zip_code_prefix_customer,geolocation_lat_customer,geolocation_lng_customer,review_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp,payment_sequential,payment_type,payment_installments,payment_value,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,geolocation_zip_code_prefix_seller,geolocation_lat_seller,geolocation_lng_seller,seller_zip_code_prefix,seller_city,seller_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,3149.0,-23.574809,-46.587471,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11,2017-10-12 03:43:48,1.0,credit_card,1.0,18.12,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,-23.680114,-46.452454,9350.0,maua,SP
1,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,3149.0,-23.574809,-46.587471,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11,2017-10-12 03:43:48,3.0,voucher,1.0,2.00,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,-23.680114,-46.452454,9350.0,maua,SP
2,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18,7c396fd4830fd04220f754e42b4e5bff,3149,sao paulo,SP,3149.0,-23.574809,-46.587471,a54f0611adc9ed256b57ede6b6eb5114,4,NaN,"Não testei o produto ainda, mas ele veio corre...",2017-10-11,2017-10-12 03:43:48,2.0,voucher,1.0,18.59,1.0,87285b34884572647811a353c7ac498a,3504c0cb71d7fa48d967e0e4c94d59d9,2017-10-06 11:07:15,29.99,8.72,utilidades_domesticas,40.0,268.0,4.0,500.0,19.0,8.0,13.0,9350.0,-23.680114,-46.452454,9350.0,maua,SP
3,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13,af07308b275d755c9edb36a90c618231,47813,barreiras,BA,47813.0,-12.169860,-44.988369,8d5266042046a06655c8db133d120ba5,4,Muito boa a loja,Muito bom o produto.,2018-08-08,2018-08-08 18:37:50,1.0,boleto,1.0,141.46,1.0,595fac2a385ac33a80bd5114aec74eb8,289cdb325fb7e7f891c38608bf9e0962,2018-07-30 03:24:27,118.70,22.76,perfumaria,29.0,178.0,1.0,400.0,19.0,13.0,19.0,31570.0,-19.810119,-43.984727,31570.0,belo horizonte,SP
4,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04,3a653a41f6f9fc3d2a113cf8398680e8,75265,vianopolis,GO,75265.0,-16.746337,-48.514624,e73b67b67587f7644d5bd1a52deb1b01,5,NaN,NaN,2018-08-18,2018-08-22 19:07:58,1.0,credit_card,3.0,179.12,1.0,aa4383b373c6aca5d8797843e5594415,4869f7a5dfa277a7dca6462dcf3b52b2,2018-08-13 08:55:23,159.90,19.22,automotivo,46.0,232.0,1.0,420.0,24.0,19.0,21.0,14840.0,-21.362358,-48.232976,14840.0,guariba,SP


In [65]:
brazilian_ecommerce = ord_cust_geo_rev_pay_item_prod_sell.copy()

In [66]:
brazilian_ecommerce.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 119151 entries, 0 to 119150
Data columns (total 45 columns):
 #   Column                                Non-Null Count   Dtype         
---  ------                                --------------   -----         
 0   order_id                              119151 non-null  object        
 1   customer_id                           119151 non-null  object        
 2   order_status                          119151 non-null  object        
 3   order_purchase_timestamp              119151 non-null  datetime64[ns]
 4   order_approved_at                     118974 non-null  datetime64[ns]
 5   order_delivered_carrier_date          117065 non-null  datetime64[ns]
 6   order_delivered_customer_date         115730 non-null  datetime64[ns]
 7   order_estimated_delivery_date         119151 non-null  datetime64[ns]
 8   customer_unique_id                    119151 non-null  object        
 9   customer_zip_code_prefix              119151 non-null  int6

In [67]:
brazilian_ecommerce['order_item_id'].unique()

array([ 1.,  2.,  3.,  4.,  5.,  6., nan,  7.,  8.,  9., 10., 11., 12.,
       13., 14., 15., 16., 17., 18., 19., 20., 21.])

In [68]:
brazilian_ecommerce.isnull().sum()

order_id                                     0
customer_id                                  0
order_status                                 0
order_purchase_timestamp                     0
order_approved_at                          177
order_delivered_carrier_date              2086
order_delivered_customer_date             3421
order_estimated_delivery_date                0
customer_unique_id                           0
customer_zip_code_prefix                     0
customer_city                                0
customer_state                               0
geolocation_zip_code_prefix_customer       322
geolocation_lat_customer                   322
geolocation_lng_customer                   322
review_id                                    0
review_score                                 0
review_comment_title                    104962
review_comment_message                   67901
review_creation_date                         0
review_answer_timestamp                      0
payment_seque

In [69]:
# dropping redundant columns
brazilian_ecommerce.drop(['geolocation_zip_code_prefix_seller', 'geolocation_zip_code_prefix_customer'],
                         axis=1, inplace=True)

In [70]:
# saving merged ecommerce dataset
# only saving when invoked as script
if __name__ != "__main__":
    brazilian_ecommerce.to_csv('../resources/Brazilian_retail/brazilian_ecommerce1.csv', index=False, 
                               encoding='utf-16')

In [71]:
# loading dataset for test purpose
df = pd.read_csv('../resources/Brazilian_retail/brazilian_ecommerce1.csv', 
                 encoding='utf-16', lineterminator='\n')

In [72]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 119151 entries, 0 to 119150
Data columns (total 43 columns):
 #   Column                         Non-Null Count   Dtype  
---  ------                         --------------   -----  
 0   order_id                       119151 non-null  object 
 1   customer_id                    119151 non-null  object 
 2   order_status                   119151 non-null  object 
 3   order_purchase_timestamp       119151 non-null  object 
 4   order_approved_at              118974 non-null  object 
 5   order_delivered_carrier_date   117065 non-null  object 
 6   order_delivered_customer_date  115730 non-null  object 
 7   order_estimated_delivery_date  119151 non-null  object 
 8   customer_unique_id             119151 non-null  object 
 9   customer_zip_code_prefix       119151 non-null  int64  
 10  customer_city                  119151 non-null  object 
 11  customer_state                 119151 non-null  object 
 12  geolocation_lat_customer      